# Record Linkage
A demo of the excellent [Record Linkage](https://recordlinkage.readthedocs.io/en/latest/) toolkit specific to the challenges expected when merging health, police and emergency shelter data.

In [1]:
import pandas as pd
import numpy as np
import recordlinkage
from recordlinkage.datasets import load_febrl1

In [3]:
dfA = load_febrl1()

In [6]:
dfA.loc['rec-233-org']

given_name                 lucas
surname                    hearn
street_number                 22
address_1        crofts crescent
address_2                    NaN
suburb                  bonbeach
postcode                    3393
state                         sa
date_of_birth           19240909
soc_sec_id               1762633
Name: rec-233-org, dtype: object

In [8]:
dfA.loc['rec-233-dup-0']

given_name                  lucad
surname                     hearn
street_number                  22
address_1        crofts crryscent
address_2                     NaN
suburb                   bonbeach
postcode                     3393
state                          sa
date_of_birth            19240909
soc_sec_id                1762633
Name: rec-233-dup-0, dtype: object

In [4]:
dfA

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933
...,...,...,...,...,...,...,...,...,...,...
rec-188-dup-0,stephanie,geu,28,bainton crescent,masonic memorial village,maryborough,2541,sa,19421008,3997529
rec-334-dup-0,nicholas,NaN,289,britten-jonues drive,jabaru court,paddington,2000,vic,19970422,5062738
rec-469-dup-0,lachlan,katsiavos,29,paul coe cdrescent,NaN,casual,2913,nsw,19380406,4112327
